In [82]:
import requests
from bs4 import BeautifulSoup
import time
import csv

In [83]:
need_reviews_cnt = 10000
reviews = []
review_data=[]
authors=[]

제목,본문,평점 코드

In [84]:
######################################
#### 영화제목, 리뷰, 평점 crawling ###
######################################

#page를 1부터 1씩 증가하며 URL을 다음 페이지로 바꿈 
for page in range(1,100): #페이지값 지정해주기: 임의로 100p로 지정
    url = f'https://movie.naver.com/movie/point/af/list.naver?&page={page}'
    #get : request로 url의  html문서의 내용 요청
    html = requests.get(url)
    #html을 받아온 문서를 .content로 지정 후 soup객체로 변환
    soup = BeautifulSoup(html.content,'html.parser')
    #find_all : 지정한 태그의 내용을 모두 찾아 리스트로 반환
    reviews = soup.find_all("td",{"class":"title"})
    
    #한 페이지의 리뷰 리스트의 리뷰를 하나씩 보면서 데이터 추출
    for review in reviews:
        sentence = review.find("a",{"class":"report"}).get("onclick").split("', '")[2]
        #만약 리뷰 내용이 비어있다면 데이터를 사용하지 않음
        if sentence != "":
            movie = review.find("a",{"class":"movie color_b"}).get_text()
            score = review.find("em").get_text()
            review_data.append([movie,sentence,int(score)])
            need_reviews_cnt-= 1
    #현재까지 수집된 리뷰가 목표 수집 리뷰보다 많아진 경우 크롤링 중지        
    if need_reviews_cnt < 0:                                         
        break
    #다음 페이지를 조회하기 전 0.5초 시간 차를 두기
    time.sleep(0.2)
     
columns_name = ["movie","sentence","score"]
with open ( "naver_movie_content.csv", "w", newline ="",encoding = 'utf8' ) as f:
    write = csv.writer(f)
    write.writerow(columns_name)
    write.writerows(review_data)

In [85]:
import pandas as pd

review = pd.read_csv('naver_movie_content.csv')
review

,movie,sentence,score
0,모비우스,믿고보는 마블스토리 ㅋ,10
1,블랙라이트,액션을 기대하고 봤다면 반대. 연기를 기대하고 봤다면 끝까지,7
2,모비우스,사전정보 없이 봤는데 러닝타임 내내 시간 가는 줄 모르게 봤습니다,10
3,모비우스,미쳤다.. 내 시간 순삭 당하고 옴,10
4,모비우스,이 영화는 한마디로 헤리포터+제다이(스타워즈)+호빗+스타트렉 4가지의 영화를 합친...,10
...,...,...,...
9368,기묘한 가족,스토리도 좋고 두시간 내내웃은 영화는 이게처음이다 이런게 백만가야지,10
9369,로마의 휴일,어느포인트에서 웃길지 모르는 감독과 어느포인트에서 웃어야 할지 모르는 관객과의 팽팽...,2
9370,위크엔드 어웨이,시간 순삭이네요 재밌게 봤어요,10
9371,크루엘라,후반부가 살짝 느슨한 느낌이었지만 재밌음! 엔딩이 뭔가 싶었는데 2가 나온다고 함.,9
